<a href="https://colab.research.google.com/github/FKLC/Torrent-To-Google-Drive-Downloader/blob/master/Torrent_To_Google_Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader 

**Important Note:** To get more disk space thanks to [@kasvith](https://github.com/kasvith) [answer](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/5#issuecomment-521911452) you have to 
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Mount Google Drive
To stream files we need to mount Google Drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Upload Your Torrent File

In [0]:
from google.colab import files
source = files.upload() # or change this with magnet link like "magnet:?xt=urn:btih:c5af268ec55cf2d3b439e7311ad43101ba8322eb&tr=http%3A%2F%2Facademictorrents.com%2Fannounce.php&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969"

### Install libtorrent

In [0]:
!apt install python3-libtorrent

### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [0]:
import libtorrent as lt
import time
import sys

ses = lt.session()
ses.listen_on(6881, 6891)


params = {"save_path": "/content/drive/My Drive/Torrent"}
if isinstance(source, str):
  h = lt.add_magnet_uri(ses, source, params)
else:
  params["ti"] = lt.torrent_info(list(source.keys())[0])
  h = ses.add_torrent(params)
print("starting", h.name())

while not h.is_seed():
    s = h.status()

    state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
    ]
    print(
        "\r%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s"
        % (
            s.progress * 100,
            s.download_rate / 1000,
            s.upload_rate / 1000,
            s.num_peers,
            state_str[s.state],
        )
    )
    sys.stdout.flush()
    time.sleep(1)

print(h.name(), "complete")